<a href="https://colab.research.google.com/github/antoniotre86/causal-inference/blob/main/notebooks/Causal_Inference_with_dowhy_simulated_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demo: Causal Inference with DoWhy - Simulated Data

Demo on using [dowhy](https://github.com/microsoft/dowhy) for doing Causal Inference on some cooked up scenarios, to see its benefits and potential applications.

***

We simulate two scenarios based on a simplified version of the exercise / cholesterol example. 

* In **Scenario 1** we make *exercise* have a causal effect on *cholesterol*;
* in **Scenario 2** *exercise* does not have a causal effect on *cholesterol*.

We introduce a couple of confounding factors to make things interesting. We assume that these are the only confounding factors at play:
* *occupation* ("stormtrooper" or "radar technician")
* *age* in decades
* *healthy diet*: binary, which depends on *Occupation*

In both scenarios *exercise* and *cholesterol* are influenced by *age*, while *exercise* is influenced by *occupation* and *cholesterol* is influenced by *healthy diet* (and indirectly by *occupation* through it). 

We will see how *exercise* and *cholesterol* appear correlated in both scenarios, and with statistical significance. But using propensity score stratification we will be able to uncover the true causal effects. 

We will also apply the refutation methods to validate our results. 

---

**Inference steps**:
1. Define the causal graphical model
2. Initialise the dowhy model by passing the training data and the graphical model in string dot format
3. Run `.identify_estimand` for the dowhy model to identify the possible methods of inference (we will use "backdoor.propensity_score_stratification")
4. Run `.estimate` to obtain the estimated causal effect
5. Refute estimate by running built-in refutation methods.

--- 

We are going to use [dowhy](https://github.com/microsoft/dowhy) which takes as input:
- training data
- causal graphical model (dot format)

---


Install the required dependencies

In [ ]:
!pip install dowhy
!apt-get install graphviz
!pip install graphviz

In [ ]:
import pandas as pd
import numpy as np
import dowhy
from dowhy import CausalModel
from graphviz import Source
from scipy.stats import ttest_ind

import matplotlib.pyplot as plt
import seaborn as sns

print(pd.__version__)
print(np.__version__)
print(dowhy.__version__)

In [ ]:
pd.set_option("precision", 3)
sns.set()
sns.set_palette("Accent")

In [ ]:
# This prevents sklearn from printing a lot of warnings (copied from dowhy example notebooks)
import warnings
from sklearn.exceptions import DataConversionWarning, ConvergenceWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)
warnings.filterwarnings(action='ignore', category=UserWarning)

## Define the graphical model

One option is to define it as a string in dot format (really an option for small graphs only).

Another option is to use [dagitty.net](http://www.dagitty.net/dags.html) -- allows you to create complex graphs by point-and-click. You can then export the textual version of it, which can be converted to dot fairly easily.

We go for the first option here as the graph is simple enough.

In [ ]:
graph_s = """
digraph {
  exercise;
  cholesterol;
  age_years;
  occupation;
  healthy_diet;
  age_years -> exercise;
  age_years -> cholesterol;
  occupation -> exercise;
  occupation -> healthy_diet;
  healthy_diet -> cholesterol;
  exercise -> cholesterol;
}
"""

dag = Source(graph_s)

dag

# Scenario 1: Exercise has a causal effect on Cholesterol level

## Generate the data

We randomly generate *occupation* and *age_years* first as they don't depend on other variables. 

We then generate *exercise* based on *occupation* and *age* assuming that:
- stormtroopers exercise more than radar technicians
- people under 40 exercise more than people aged 40 or older

We generate the *healty_diet* variable based on *occupation*, assuming radar technicians have a better diet.

Finally, we generate *cholesterol* levels assuming that:
- it's lower with an *healthy diet* and *exercise*
- it's lower for people less than 40 years of *age*.

In [ ]:
np.random.seed(333)

N = 1000

data_s1 = pd.DataFrame(columns=["exercise", "cholesterol", "occupation", "age_years", "healthy_diet"])

data_s1["occupation"] = np.random.choice(["stormtrooper", "radar_technician"], N)
data_s1["age_years"] = (np.random.normal(40, 10, N) / 10).astype(int) * 10

# Stormtroopers get more exercise
data_s1["exercise"] = np.random.randn(N) \
  + 1*(data_s1["occupation"] == "stormtrooper") \
  + -1*(data_s1["occupation"] == "radar_technician")
# People under 40 get more exercise
data_s1["exercise"] += 1*(data_s1["age_years"] < 40)
data_s1["exercise"] = (data_s1["exercise"] > data_s1["exercise"].mean())


# Stormtroopers have a poorer diet
data_s1["healthy_diet"] = np.random.rand(N) < (0.1*(data_s1["occupation"] == "stormtrooper") 
                                              + 0.9*(data_s1["occupation"] == "radar_technician"))

# Base level of cholesterol (random)
data_s1["cholesterol"] = 5 + np.random.randn(N)**2 

# A healthy diet gives lower cholesterol
data_s1["cholesterol"] += -3*data_s1["healthy_diet"] 

# People under 40 have lower cholesterol
data_s1["cholesterol"] += -1*(data_s1["age_years"] < 40)

In [ ]:
# Exercise has the effect of reducing cholesterol
actual_effect_s1 = -1
data_s1["cholesterol"] += actual_effect_s1*data_s1["exercise"]

In [ ]:
data_s1

## Let's look at correlations first

Between *occupation*, *exercise* and *cholesterol*:

In [ ]:
data_s1.groupby(["occupation", "exercise"])["cholesterol"].mean().unstack(1)

As expected, *cholesterol* is the highest for stormtroopers who don't exercise, and the lowest for radar technicians who exercise.

Let's add *age* as well:

In [ ]:
data_s1.groupby(["age_years", "occupation", "exercise"])["cholesterol"].mean().unstack([1,2])


For most age levels, cholesterol is lower when they exercise, and in general it's also lower for radar technicians.

Now we calculate the overall correlation coefficients:

In [ ]:
data_s1.corr()

Most of these are expected, considering on how we generated the data. 

However, the correlation between cholesterol and exercise stands out. It looks like there is a POSITIVE correlation, meaning the more one exercise the higher their cholesterol levels!

In [ ]:
data_s1.corr().loc["cholesterol", "exercise"].round(3)

In [ ]:
data_s1.groupby("exercise")["cholesterol"].mean()

In [ ]:
observed_effect_s1 = data_s1.groupby("exercise")["cholesterol"].mean().diff().loc[True]
observed_effect_s1.round(3)

This appears to be an example of the [Simpson's Paradox](https://en.wikipedia.org/wiki/Simpson%27s_paradox)

> *Simpson's paradox, which also goes by several other names, is a phenomenon in probability and statistics in which a trend appears in several groups of data but disappears or reverses when the groups are combined.*

In our example, cholesterol is lower for people who exercise if we break down the data by occupation and age, but it appears to be positively correlated with exercise if we look at the data in aggregate.

In [ ]:
ttest_ind(data_s1.loc[data_s1["exercise"], "cholesterol"].values, data_s1.loc[~data_s1["exercise"], "cholesterol"].values)

A t-test confirms that the correlation is also statistically significant with p-value < 0.001.

## Causal Inference

We now turn to Causal Inference to hopefully find the true causal effect between exercise and cholesterol. 

We are going to use [dowhy]() for the task, and use [Propensity Score Matching](https://en.wikipedia.org/wiki/Propensity_score_matching) (with stratification) to estimate the actual causal effect. 

***

We first instantiate the model and pass our data, the treatment and outcome names, and the graph in dot format.


In [ ]:
model_s1 = CausalModel(
        data=data_s1,
        treatment="exercise",
        outcome="cholesterol",
        graph=graph_s.replace('\n', '').strip()
    )

Next, we run the actual estimation method.

`.identify_effect` parses the graph and defines the estimands.

`.estimate_effect` with "backdoor.propensity_score_stratification" as `method_name` runs the propensity score model and returns the estimated effect.

Here we can also define 
- specific method parameters in `method_params`; for PS stratification we can define the number of strata and how many items of one class required in each stratum as a minumum (otherwise the stratum is dropped);
- whether we want a confidence interval around the estimate, and the method to compute it; in this case we decided to calculate it with bootstrapping;
- whether we want to test for statistical significance (not recommended if you are calculating the confidence interval already as it will just slow down the estimation -- we can judge significance at the 95% level from the confidence interval already). 

In [ ]:
# Identify causal effect and return target estimands
identified_estimand = model_s1.identify_effect(proceed_when_unidentifiable=True)

# Estimate the target estimand using a statistical method. Propensity score stratification.
estimate_s1 = model_s1.estimate_effect(identified_estimand,
                                  method_name="backdoor.propensity_score_stratification",
                                  target_units="ate",
                                  confidence_intervals='bootstrap',
                                  test_significance=False,
                                  method_params={"num_strata":20, "clipping_threshold":5, "num_simulations":100})

estimated_effect_s1 = estimate_s1.value
estimated_effect_c1_s1 = estimate_s1.get_confidence_intervals()

print(f"Estimated causal effect: {estimated_effect_s1:0.3g}; "
  f"95% conf. int.: ({estimated_effect_c1_s1[0]:0.3g}, {estimated_effect_c1_s1[1]:0.3g})")
print(f"Actual causal effect: {actual_effect_s1}")

The estimated causal effect is correclty negative and close to the actual effect we defined when simulating the data (it's contained in the estimate confidence interval). 

What we would have thought being a positive relationship between exercise and cholesterol (more exercise - higher cholesterol), is in reality a negative one; the positive correlation we had observed was entirely due to the effect of the confounding factors. 

## Refutation

We will now see if we can refute the estimates (even though it's not necessary in this case as we have generated the data ourselves based on a pre-defined causal model).

Dowhy provides a collection of refutation methods to use. We will use three of them:
- adding a random confounder
- replacing treatment with a placebo
- re-running the estimate on data subsets

In [ ]:
np.random.seed(42)

refute_results_0 = model_s1.refute_estimate(identified_estimand, 
                                          estimate_s1, 
                                          method_name="random_common_cause", 
                                          confounders_effect_on_treatment="binary_flip", 
                                          confounders_effect_on_outcome="binary_flip")
print(refute_results_0)

In [ ]:
np.random.seed(42)

refute_results_1 = model_s1.refute_estimate(identified_estimand, 
                                         estimate_s1, 
                                         method_name="placebo_treatment_refuter", 
                                         placebo_type="Random Data", 
                                         num_simulations=10)

print(refute_results_1)

In [ ]:
np.random.seed(42)

refute_results_2 = model_s1.refute_estimate(identified_estimand, 
                                         estimate_s1, 
                                         method_name="data_subset_refuter", 
                                         subset_fraction=0.5, 
                                         num_simulations=10)

print(refute_results_2)

The three refutation methods used return the expected results:
- adding a randomly drawn confounder does not significantly change the estimate
- replacing the treatment with a randomly shuffled one makes the effect go to zero
- re-running the estimates on several subsamples of the data returns a similar estimate as the original one.

# Scenario 2: Exercise does not have an impact on Cholesterol level

Let's now see what happens if we generate the data such that there is no direct causal effect between exercise and cholesterol.

In [ ]:
np.random.seed(333)

N = 1000

data_s2 = data_s1.copy()

# Base level of cholesterol (random)
data_s2["cholesterol"] = 5 + np.random.randn(N)**2 

# A healthy diet gives lower cholesterol
data_s2["cholesterol"] += -3*data_s2["healthy_diet"] 

# People under 40 have lower cholesterol
data_s2["cholesterol"] += -1*(data_s2["age_years"] < 40)

In [ ]:
# Exercise does not affect cholesterol
actual_effect_s2 = 0.0
data_s2["cholesterol"] += actual_effect_s2*data_s2["exercise"]

## Let's look at correlations first

Between *occupation*, *exercise* and *cholesterol*:

In [ ]:
data_s2.groupby(["occupation", "exercise"])["cholesterol"].mean().unstack(1)

Like before, *cholesterol* is the highest for stormtroopers who don't exercise, and the lowest for radar technicians who exercise.

Let's add *age* as well:

In [ ]:
data_s2.groupby(["age_years", "occupation", "exercise"])["cholesterol"].mean().unstack([1,2])


Cholesterol is still higher with older age, but now we don't see a correlation with exercise in the individual age groups.

Now we calculate the overall correlation coefficients:

In [ ]:
data_s2.corr()

However, now cholesterol and exercise appear even more positively correlated!

In [ ]:
data_s2.corr().loc["cholesterol", "exercise"].round(3)

In [ ]:
data_s2.groupby("exercise")["cholesterol"].mean()

In [ ]:
observed_effect_s2 = data_s2.groupby("exercise")["cholesterol"].mean().diff().loc[True]
observed_effect_s2.round(3)

In [ ]:
ttest_ind(data_s2.loc[data_s2["exercise"], "cholesterol"].values, data_s2.loc[~data_s1["exercise"], "cholesterol"].values)

A t-test confirms that the correlation is also statistically significant with p-value < 0.001.

## Causal Inference

Let's use propensity score matching like before to see if it's able to estimate the real causal effect between exercise and cholesterol in this scenario (should be close to 0). 


In [ ]:
np.random.seed(42)

model_s2 = CausalModel(
    data=data_s2,
    treatment="exercise",
    outcome="cholesterol",
    graph=graph_s.replace('\n', '').strip()
)

# Identify causal effect and return target estimands
identified_estimand = model_s2.identify_effect(proceed_when_unidentifiable=True)

# Estimate the target estimand using a statistical method. Propensity score stratification.
estimate_s2 = model_s2.estimate_effect(identified_estimand,
                                    method_name="backdoor.propensity_score_stratification",
                                    target_units="ate",
                                    test_significance=False,
                                    confidence_intervals='bootstrap',
                                    method_params={"num_strata":20, "clipping_threshold":5, "num_simulations":100})

estimated_effect_s2 = estimate_s2.value
estimated_effect_c1_s2 = estimate_s2.get_confidence_intervals()

print(f"Estimated causal effect: {estimated_effect_s2:0.3g}; "
  f"95% conf. int.: ({estimated_effect_c1_s2[0]:0.3g}, {estimated_effect_c1_s2[1]:0.3g})")
print(f"Actual causal effect: {actual_effect_s2}")

Indeed, the estimated causal effect is correclty close to zero (confirmed also by the confidence interval including zero).

Again, what we would have thought being a positive relationship between exercise and cholesterol (and this time even a stronger one than in the first scenario), does not exist in reality; the positive correlation we had observed was entirely due to the effect of the confounding factors. 

# Analysis of Propensity Score model (back to Scenario 1)

We can look into how the propensity score model is performing in the background.

In [ ]:
treatment_name = "exercise"
outcome_name = "cholesterol"

model = model_s1
estimate = estimate_s1

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))

ax.set_title("Distribution of propensity scores")
ax.set_xlabel("Propensity score")
df_ = pd.DataFrame({"p": estimate.propensity_scores, treatment_name:data_s1[treatment_name]})
for t in df_[treatment_name].unique():
    sns.kdeplot(df_.loc[df_[treatment_name] == t, "p"], ax=ax, label=f"{treatment_name}={t}", clip=(0,1))
ax.set_xlim(0,1)
ax.legend()

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))

ax.set_title("Distribution of estimated ATE")
ax.set_xlabel("ATE estimate")
ax.hist(estimate.estimator._bootstrap_estimates.estimates, bins=20, density=True)
sns.kdeplot(estimate.estimator._bootstrap_estimates.estimates, ax=ax, color='lightblue')
cil, cih = estimate.get_confidence_intervals()
ax.axvline(estimate.value, color='slategrey')
ax.axvline(cil, linestyle='dashed', color='slategrey')
ax.axvline(cih, linestyle='dashed', color='slategrey')
ymin, ymax = ax.get_ylim()
ax.text(estimate.value, ymax, f" mean: {estimate.value:0.3f}", color='slategrey', va='top', size=9)
ax.text(cil, ymax, f" CI-L: {cil:0.3f}", color='slategrey', va='top', size=9)
ax.text(cih, ymax, f" CI-H: {cih:0.3f}", color='slategrey', va='top', size=9)
# Plot x=0 line if x=0 in view already
xmin, xmax = ax.get_xlim()
if np.sign(xmin*xmax) == -1:
    ax.axvline(0, color='firebrick')

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score


fig, ax = plt.subplots(figsize=(10,8))

fpr, tpr, _ = roc_curve(data_s1[treatment_name].values*1, estimate.propensity_scores, pos_label=1)
roc_score = roc_auc_score(data_s1[treatment_name].values*1, estimate.propensity_scores)
ax.plot(fpr, tpr, '-.')
ax.plot([0, 1], [0, 1], 'r--')
ax.text(0.8, 0.2, f"AUC={roc_score:.3f}", ha='center', color='slategrey')
ax.set_xlabel("FPR")
ax.set_ylabel("TPR")
ax.set_title(f"Propensity score estimator ROC")

In [ ]:
from sklearn.metrics import precision_recall_curve

fig, ax = plt.subplots(figsize=(10,8))

p, r, _ = precision_recall_curve(data_s1[treatment_name].values*1, estimate.propensity_scores, pos_label=1)
ax.plot(p, r, '-.')
ax.set_xlabel("Precision")
ax.set_ylabel("Recall")
ax.set_title(f"Propensity score estimator P/R curve")

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))

coef_ = estimate.estimator._propensity_score_model.coef_[0]
coef_weights = (estimate.estimator._observed_common_causes.mean().T / estimate.estimator._observed_common_causes.mean().sum()).values
fnames = estimate.estimator._observed_common_causes.columns
ax = pd.DataFrame({'coef_weighted':coef_/coef_weights}, index=fnames).sort_values('coef_weighted')\
    .plot(kind='barh', legend=False, ax=ax)
ax.set_title('Propensity score estimator feature importance')
ax.set_xlabel('Feature importance')
ax.set_ylabel('Propensity model feature')

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))


def _strata_counts(df, treatment_name):
    df_strata_counts_1 = df.groupby(['strata', treatment_name])[treatment_name].count().unstack(1).reset_index()
    df_strata_counts_1.columns = ['stratum'] + [f"{o}" for o in df[treatment_name].unique()]
    df_strata_counts_2 = df_strata_counts_1.set_index('stratum').stack().reset_index()
    df_strata_counts_2.columns = ['stratum', treatment_name, 'count']
    df_strata_counts_2['stratum'] = df_strata_counts_2['stratum'].astype(int)

    return df_strata_counts_2


def _effect_per_strata(df, treatment_name):
    df_strata_effect_1 = df_.groupby('strata').agg({
                    treatment_name: ['sum'],
                    'dbar': ['sum'],
                    'd_y': ['sum'],
                    'dbar_y': ['sum']
                }).reset_index().rename(columns={'strata':'stratum'}).set_index('stratum')
    df_strata_effect_1.columns = ["_".join(x) for x in df_strata_effect_1.columns.ravel()]
    treatment_sum_name = treatment_name + "_sum"
    control_sum_name = "dbar_sum"
    df_strata_effect_1['d_y_mean'] = df_strata_effect_1['d_y_sum'] / df_strata_effect_1[treatment_sum_name]
    df_strata_effect_1['dbar_y_mean'] = df_strata_effect_1['dbar_y_sum'] / df_strata_effect_1['dbar_sum']
    df_strata_effect_1['effect'] = df_strata_effect_1['d_y_mean'] - df_strata_effect_1['dbar_y_mean']
    df_strata_effect_1 = df_strata_effect_1['effect'].reset_index()

    return df_strata_effect_1


df_ = estimate.estimator._data
df_strata_counts = _strata_counts(df_, treatment_name)
df_strata_effect = _effect_per_strata(df_, treatment_name)

sns.barplot(x='stratum', y='count', data=df_strata_counts, hue=treatment_name, ax=ax)
ax_secondary = ax.twinx()
ax_secondary.set_ylabel('effect')
df_strata_effect['effect'].plot(ax=ax_secondary, grid=False, linewidth=2, color='gold', marker='*', label='effect')
l_scale = lambda x, factor: x*(1+np.sign(x)*factor)
ax_secondary.set_ylim(None, l_scale(ax_secondary.get_ylim()[1], .3))

ax.set_title('Propensity score stratification - strata counts and effects')